<h1 style="color:Orange;text-align:center; font-size:40px;">  Web Scraping - Mission to Mars</h1>

### Importing dependencies

In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
from webdriver_manager.chrome import ChromeDriverManager
import urllib.request
from selenium import webdriver

<h1 style="color:blue;text-align:center; font-size:40px;"> Scraping </h1>

<h2 style="color:green;text-align:center;"> NASA Mars News </h2>

In [2]:
# Mars News site
url = "https://redplanetscience.com/"

In [3]:
driver = webdriver.Chrome()
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()
print(soup.prettify())

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
# Retrieve page with the requests module
response = requests.get(url)
# response = urllib.request.urlopen(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
soup

In [ ]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='list_text')
print(results)